# Wiki dump xml parser

## Preparations
* just download the [ruwiki-latest-pages-articles.xml.bz2](https://dumps.wikimedia.org/ruwiki/latest/ruwiki-latest-pages-articles.xml.bz2) file 
* open Python 3 and lets begin

## Programming

* Import `BZ2` archive reader:

In [1]:
from bz2 import BZ2File

* Import `ElementTree` from `xml` lib:

In [2]:
import xml.etree.ElementTree as etree

* Lets add function for stripping tag names:

In [3]:
def strip_tag_name(t):
    idx = k = t.rfind("}")
    if idx != -1:
        return t[idx + 1:]
    else:
        return t

* Now we create generator-like reader for our `bz2` wiki dump:

In [4]:
def read_wiki_dump(bz2_dump_path):
    with BZ2File(bz2_dump_path) as xml_file:
        for event, elem in etree.iterparse(xml_file, events=("start", "end")):
            tname = strip_tag_name(elem.tag)
            if event == "start":
                # We will read only "page" nodes
                if tname == "page":
                    # Init necessary fields
                    title = ""
                    redirect = ""
                    ns = 0
                    text = ""
            else:
                # Assign fields values
                if tname == "title":
                    title = elem.text
                elif tname == "redirect":
                    redirect = elem.attrib["title"]
                elif tname == "ns":
                    ns = int(elem.text)
                elif tname == "text":
                    text = elem.text
                elif tname == "page":
                    # Yield fields
                    yield title, redirect, ns, text
                elem.clear()

* OK. Lets read littlebit of the dump data:

In [5]:
from itertools import islice

XML_BZ2_FILE_PATH = "J:\\ruwiki-latest-pages-articles.xml.bz2"
SLICE_SIZE = 5

for title, redirect, ns, text in islice(read_wiki_dump(XML_BZ2_FILE_PATH), SLICE_SIZE):
    print("title: {}\r\nredirect: {}\r\nns: {}\r\ntext slice (first 50 chars): {}".format(title, redirect, ns, text[:50]))

title: Базовая статья
redirect: Заглавная страница
ns: 0
text slice (first 50 chars): #REDIRECT [[Заглавная страница]]
title: Литва
redirect: 
ns: 0
text slice (first 50 chars): {{другие значения}}
{{Государство
 |Русское назван
title: Россия
redirect: 
ns: 0
text slice (first 50 chars): {{Другие значения}}
{{Перенаправление|РФ}}
{{Госуд
title: Слоновые
redirect: 
ns: 0
text slice (first 50 chars): {{перенаправление|Слон}}
{{Таксон
|image file=Afri
title: Мамонты
redirect: 
ns: 0
text slice (first 50 chars): {{перенаправление|Мамонт}}
{{Таксон
 | image file 


Lets see `siteinfo` section:
```
      <namespace key="-2" case="first-letter">Медиа</namespace>
      <namespace key="-1" case="first-letter">Служебная</namespace>
      <namespace key="0" case="first-letter" />
      <namespace key="1" case="first-letter">Обсуждение</namespace>
      <namespace key="2" case="first-letter">Участник</namespace>
      <namespace key="3" case="first-letter">Обсуждение участника</namespace>
      <namespace key="4" case="first-letter">Википедия</namespace>
      <namespace key="5" case="first-letter">Обсуждение Википедии</namespace>
      <namespace key="6" case="first-letter">Файл</namespace>
      <namespace key="7" case="first-letter">Обсуждение файла</namespace>
      <namespace key="8" case="first-letter">MediaWiki</namespace>
      <namespace key="9" case="first-letter">Обсуждение MediaWiki</namespace>
      <namespace key="10" case="first-letter">Шаблон</namespace>
      <namespace key="11" case="first-letter">Обсуждение шаблона</namespace>
      <namespace key="12" case="first-letter">Справка</namespace>
      <namespace key="13" case="first-letter">Обсуждение справки</namespace>
      <namespace key="14" case="first-letter">Категория</namespace>
      <namespace key="15" case="first-letter">Обсуждение категории</namespace>
      <namespace key="100" case="first-letter">Портал</namespace>
      <namespace key="101" case="first-letter">Обсуждение портала</namespace>
      <namespace key="102" case="first-letter">Инкубатор</namespace>
      <namespace key="103" case="first-letter">Обсуждение Инкубатора</namespace>
      <namespace key="104" case="first-letter">Проект</namespace>
      <namespace key="105" case="first-letter">Обсуждение проекта</namespace>
      <namespace key="106" case="first-letter">Арбитраж</namespace>
      <namespace key="107" case="first-letter">Обсуждение арбитража</namespace>
      <namespace key="446" case="first-letter">Образовательная программа</namespace>
      <namespace key="447" case="first-letter">Обсуждение образовательной программы</namespace>
      <namespace key="828" case="first-letter">Модуль</namespace>
      <namespace key="829" case="first-letter">Обсуждение модуля</namespace>
      <namespace key="2300" case="first-letter">Gadget</namespace>
      <namespace key="2301" case="first-letter">Gadget talk</namespace>
      <namespace key="2302" case="case-sensitive">Gadget definition</namespace>
      <namespace key="2303" case="case-sensitive">Gadget definition talk</namespace>
      <namespace key="2600" case="first-letter">Тема</namespace>
```
Our `ns` field equals `namespace key`.

* OK. Lets filter our database and print only `articles`:

In [6]:
SLICE_SIZE = 10

for title, redirect, ns, text in islice(
        # Lambda can get only one argument (here is tuple(title, redirect, ns, text)) it[2] -- is a "ns", and it[1] -- is a redirect
        filter(lambda it: it[2] == 0 and len(it[1]) == 0, 
               read_wiki_dump(XML_BZ2_FILE_PATH)), 
    SLICE_SIZE):
    print(title)

Литва
Россия
Слоновые
Мамонты
Красная книга
Соционика
Школа
Лингвистика
Социология
Киевская Русь
